In [1]:
!pip install azure-storage-blob


In [4]:
import os, uuid, glob
from azure.storage.blob import BlobServiceClient, BlobClient, ContentSettings, ContainerClient, __version__
import subprocess

In [ ]:
# replace this connection string and container name with your storage connection and container name 
connect_str="BlobEndpoint=https:endpoint"
container_name="tempaudio"
wavcontainer_name='tempwav'

In [6]:
# Replace with the local folder where you want files to be downloaded
local_folder_path = "sampledata/wavoutput/"


# %%


class AzureBlobFileDownloader:
  def __init__(self):
    print("Intializing AzureBlobFileDownloader")
 
    # Initialize the connection to Azure storage account
    self.blob_service_client =  BlobServiceClient.from_connection_string(connect_str)
    self.my_container = self.blob_service_client.get_container_client(container_name)
    #self.wav_container= self.blob_service_client.get_container_client(wavcontainer_name)
 
 
  def save_blob(self,file_name,file_content):
    # Get full path to the file
    download_file_path = os.path.join(local_folder_path, file_name)
 
    # for nested blobs, create local path as well!
    os.makedirs(os.path.dirname(download_file_path), exist_ok=True)
 
    with open(download_file_path, "wb") as file:
      file.write(file_content)
    
    #filepath='sampledata/mp4input/call-center-talk-video-sample.mp4'
    download_file_path_no_ext = os.path.splitext(download_file_path)[0]
    command = "ffmpeg -i"+" "+download_file_path+" "+"-ar 16000 -vn"+" "+download_file_path_no_ext+".wav"
    print('command :',command)
    subprocess.call(command, shell=True)
    
    # Create a blob client using the local file name as the name for the blob
    wav_local_file_name=download_file_path_no_ext+".wav"
    blob_client = self.blob_service_client.get_blob_client(container=wavcontainer_name, blob=wav_local_file_name)

    print("\nUploading wav file to Azure Storage as blob:\n\t" + wav_local_file_name)

    # Upload the created file
    with open(wav_local_file_name, "rb") as data:
        blob_client.upload_blob(data)
        
 
  def download_blob(self):
    my_blobs = self.my_container.list_blobs()
    for blob in my_blobs:
      print(blob.name)
      bytes = self.my_container.get_blob_client(blob).download_blob().readall()
      self.save_blob(blob.name, bytes)
      #self.my_container.get_blob_client(blob).delete_blob()
        

        
        
 
# Initialize class and upload files
azure_blob_file_downloader = AzureBlobFileDownloader()
azure_blob_file_downloader.download_blob()


Intializing AzureBlobFileDownloader
Clip1.mp3
command : ffmpeg -i sampledata/wavoutput/Clip1.mp3 -ar 16000 -vn sampledata/wavoutput/Clip1.wav

Uploading wav file to Azure Storage as blob:
	sampledata/wavoutput/Clip1.wav
Clip2.mp3
command : ffmpeg -i sampledata/wavoutput/Clip2.mp3 -ar 16000 -vn sampledata/wavoutput/Clip2.wav

Uploading wav file to Azure Storage as blob:
	sampledata/wavoutput/Clip2.wav
call-center-talk-video-sample.mp4
command : ffmpeg -i sampledata/wavoutput/call-center-talk-video-sample.mp4 -ar 16000 -vn sampledata/wavoutput/call-center-talk-video-sample.wav

Uploading wav file to Azure Storage as blob:
	sampledata/wavoutput/call-center-talk-video-sample.wav


In [7]:
#empty local folder
import os, shutil
folder_tbd = local_folder_path
for filename in os.listdir(folder_tbd):
    file_path = os.path.join(folder_tbd, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))